In [1]:
import sys
from PyQt5.QtWidgets import *
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
import requests
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import scipy.optimize
from sympy import exp
from scipy import optimize
from sympy import *
import matplotlib.pyplot as plt
import scipy.interpolate
from scipy import interpolate
from scipy.optimize import fmin

# 채권종류 선택 리스트
gen_req_url = 'http://www.kofiabond.or.kr/proframeWeb/XMLSERVICES/'
form_data_bnd = '''
                <message>
                  <proframeHeader>
                    <pfmAppName>BIS-KOFIABOND</pfmAppName>
                    <pfmSvcName>BISBndSrtPrcSrchSO</pfmSvcName>
                    <pfmFnName>selectBndTyp</pfmFnName>
                  </proframeHeader>
                  <systemHeader></systemHeader>
                    <BISBndSrtPrcTrmDTO>
                </BISBndSrtPrcTrmDTO>
                </message>
                '''

r = requests.post(gen_req_url, form_data_bnd)
soup = BeautifulSoup(r.content, 'lxml-xml')
col_dt = {
    'val1': 'bond_cd',
    'val2': 'bond_nm',
    'val3': 'bond_snm',
    'val4': 'bond_enm',
    'val5': 'gyn'
}
res_li = soup.find_all('BISBndSrtPrcTrmDTO')

df = pd.DataFrame()
for r in res_li:
    dt = dict()
    for i in list(col_dt.keys()):
        dt[i] = [r.find(i).text]
    df_sub = pd.DataFrame.from_dict(dt)
    df = df.append(df_sub, sort=True)

df.reset_index(drop=True, inplace=True)
df = df.rename(columns=col_dt)
bnd_df = df.reindex(columns=list(col_dt.values()))

# 채권만기 선택 리스트
form_data_mat = '''
                <message>
                  <proframeHeader>
                    <pfmAppName>BIS-COM</pfmAppName>
                    <pfmSvcName>BISComCommonCodeSO</pfmSvcName>
                    <pfmFnName>selectCommonCode</pfmFnName>
                  </proframeHeader>
                  <systemHeader></systemHeader>
                    <BISComCommonCodeDTO>
                    <vGrpCdList>'C8002'</vGrpCdList>
                </BISComCommonCodeDTO>
                </message>
                '''
r = requests.post(gen_req_url, form_data_mat)
soup = BeautifulSoup(r.content, 'lxml-xml')
col_dt = {
    'groupCd': 'mat_gcd',
    'commonCd': 'mat_cd',
    'codeNm': 'mat_nm',
    'codeEngNm': 'mat_enm',
    'defaultYN': 'dyn',
}

res_li = soup.find_all('BISComCommonCodeDTO')

df = pd.DataFrame()
for r in res_li:
    dt = dict()
    for i in list(col_dt.keys()):
        dt[i] = [r.find(i).text]
    df_sub = pd.DataFrame.from_dict(dt)
    df = df.append(df_sub, sort=True)

df.reset_index(drop=True, inplace=True)
df = df.rename(columns=col_dt)
mat_df = df.reindex(columns=list(col_dt.values()))

# 전역변수 설정값들
YTMs = []
Maturity = [0.25, 0.5, 0.75, 1, 1.5, 2, 2.5, 3, 4, 5, 7, 10]
Bond_data = {}
column_idx_lookup = {}
column_idx_lookup_2 = {}
final_df = pd.DataFrame()
fr_date = ''
to_date = ''

class BootstrappingDialog(QDialog):

    def __init__(self):
        super().__init__()

        self.table()

    def table(self):
        self.setGeometry(800, 200, 1200, 500)

        ##### Maturity YTMs의 표를 만든다.#####

        self.tableWidget = QTableWidget(self)
        self.tableWidget.setRowCount(12)
        self.tableWidget.setColumnCount(2)
        self.tableWidget.setEditTriggers(QAbstractItemView.NoEditTriggers)
        self.tableWidget.setHorizontalHeaderLabels(['Maturity', 'YTMs'])

        for k, v in Bond_data.items():
            col = column_idx_lookup[k]
            for row, val in enumerate(v):
                item = QTableWidgetItem(val)

                self.tableWidget.setItem(row, col, item)

        self.tableWidget.resizeColumnsToContents()
        self.tableWidget.resizeRowsToContents()

        ###### 버튼을 만든다 #####

        self.Button = QPushButton("Bootstrap", self)
        self.Button.clicked.connect(self.Button_clicked)

        self.Button1 = QPushButton("Interpolation and Extrapolation", self)
        self.Button1.clicked.connect(self.Button_clicked1)

        self.Button2 = QPushButton("NSS Model", self)
        self.Button2.clicked.connect(self.Button_clicked2)

        ###### zerorates을 추가한 표를 만든다.#########

        self.tableWidget2 = QTableWidget(self)
        self.tableWidget2.setRowCount(12)
        self.tableWidget2.setColumnCount(3)
        self.tableWidget2.setHorizontalHeaderLabels(['Mat', 'YTMs', 'Zerorates'])

        for k, v in Bond_data.items():
            col = column_idx_lookup[k]
            for row, val in enumerate(v):
                item = QTableWidgetItem(val)

                self.tableWidget2.setItem(row, col, item)

        self.tableWidget2.resizeColumnsToContents()
        self.tableWidget2.resizeRowsToContents()

        ###### Interpolation 표를 만든다   ######

        self.tableWidget3 = QTableWidget(self)
        self.tableWidget3.setRowCount(120)
        self.tableWidget3.setColumnCount(2)
        self.tableWidget3.setHorizontalHeaderLabels(['Mat', 'Zerorates'])

        for k, v in Bond_data.items():
            col = 0
            for row in range(0, 120):
                i = '%s' % (row + 1)
                item = QTableWidgetItem(i)
                self.tableWidget3.setItem(row, col, item)

        self.tableWidget3.resizeColumnsToContents()
        self.tableWidget3.resizeRowsToContents()

        ####### NSS Fitting Parameter와 그래프를 만든다 ######

        self.tableWidget4 = QTableWidget(self)
        self.tableWidget4.setRowCount(1)
        self.tableWidget4.setColumnCount(6)
        self.tableWidget4.setHorizontalHeaderLabels(['beta0', 'beta1', 'beta2', 'beta3', 'tau0', 'tau1'])

        self.tableWidget4.resizeColumnsToContents()
        self.tableWidget4.resizeRowsToContents()

        self.Button3 = QPushButton("Graph", self)
        self.Button3.clicked.connect(self.Button_clicked3)

        self.fig = plt.Figure()
        self.canvas = FigureCanvas(self.fig)

        # 1번 위젯만들기
        ##### 레이아웃을 만든다 #####
        leftLayout = QVBoxLayout()  # 2번 레이아웃 매니저 만들기
        leftLayout.addWidget(self.tableWidget)  # 3번 addWidget메서드를 이용한 위젯등록
        leftLayout.addWidget(self.Button)

        rightLayout = QVBoxLayout()
        rightLayout.addWidget(self.tableWidget2)
        rightLayout.addWidget(self.Button1)
        rightLayout.addWidget(self.Button2)

        rightLayout2 = QVBoxLayout()
        rightLayout2.addWidget(self.tableWidget3)

        rightLayout3 = QVBoxLayout()
        rightLayout3.addWidget(self.tableWidget4)
        rightLayout3.addWidget(self.Button3)
        rightLayout3.addWidget(self.canvas)

        layout = QHBoxLayout()
        layout.addLayout(leftLayout)
        layout.addLayout(rightLayout)
        layout.addLayout(rightLayout2)
        layout.addLayout(rightLayout3)

        self.setLayout(layout)  # 4번 레이아웃 매니저 등록 단계

    def Button_clicked(self):
        ############# 저장되어있는 zerorates를 Bond_data_2에 저장한다 ##########
        Bond_data_2 = {
            'Zerorates': ["%s" % a[0], "%s" % a[1], "%s" % a[2], "%s" % a[3], "%s" % a[4], "%s" % a[5], "%s" % a[6],
                          "%s" % a[7], "%s" % a[8], "%s" % a[9], "%s" % a[10], "%s" % a[11]]}
        for k, v in Bond_data_2.items():
            col = column_idx_lookup_2[k]
            for row, val in enumerate(v):
                item = QTableWidgetItem(val)

                self.tableWidget2.setItem(row, col, item)
        self.tableWidget2.resizeColumnsToContents()
        self.tableWidget2.resizeRowsToContents()

    def Button_clicked1(self):
        # Interpolation 표를 채운다
        x = np.linspace(3, 120, 40)
        y = np.array(b[r])
        # x_new = np.linspace(3,120,118)
        # f = interpolate.interp1d(x,y)
        # f(x_new)
        y_interp = scipy.interpolate.interp1d(x, y)
        for i in range(3, 121):
            c.loc[i - 1, r] = y_interp(i)
        data1 = np.array([[3, c.loc[2, r]], [4, c.loc[3, r]]])
        fit = np.polyfit(data1[:, 0], data1[:, 1], 1)
        line = np.poly1d(fit)
        new_points = np.arange(2) + 1
        c.loc[0, r] = np.array(line(new_points)[0])
        c.loc[1, r] = np.array(line(new_points)[1])
        # for i in range(0,12):
        #     a.loc[i,r] = zerorates[i]
        # a = a.rename(columns={(r):data.loc[r,'일자']})

        k = []
        for i in range(0, 120):
            k.append("%s" % c.loc[i, 1])
        Bond_data_3 = {'Zerorates': k}
        for k, v in Bond_data_3.items():
            col = 1
            for row, val in enumerate(v):
                item = QTableWidgetItem(val)
                self.tableWidget3.setItem(row, col, item)
        self.tableWidget3.resizeColumnsToContents()
        self.tableWidget3.resizeRowsToContents()

    def Button_clicked2(self):
        # NSS 표를 채운다

        n = []
        for i in range(0, 6):
            n.append("%s" % m[i])
        Nss_parameter = {'beta0': n[0], 'beta1': n[1], 'beta2': n[2], 'beta3': n[3], 'tau0': n[4], 'tau1': n[5]}
        for k, v in Nss_parameter.items():
            row = 0
            for col in range(0, 6):
                item = QTableWidgetItem(n[col])

                self.tableWidget4.setItem(row, col, item)
        self.tableWidget4.resizeColumnsToContents()
        self.tableWidget4.resizeRowsToContents()

    def Button_clicked3(self):
        # QMessageBox.about(self,"message","Graph")
        ax = self.fig.add_subplot(111)
        y = [f(m, v) for v in M]

        ax.plot(M, y)
        ax.scatter(M, y_data)

        self.canvas.draw()


class MyWindow(QWidget):
    def __init__(self):
        super().__init__()
        self.setupUI()

    def setupUI(self):
        # mainwindow 크기
        self.setGeometry(800, 200, 300, 200)

        # 그룹박스
        groupBox = QGroupBox("KOFIA BOND", self)
        # groupBox.move(10, 10)
        #groupBox.resize(300, 400)

        # 조회 시작일,종료일
        label1 = QLabel("조회 시작일: ", self)
        # label1.move(10, 20)
        label2 = QLabel("조회 종료일: ", self)
        # label2.move(10, 45)

        # 조회일 설정
        self.dateBox1 = QSpinBox(self)
        self.dateBox1.setMaximum(20201201)
        # self.dateBox1.move(80, 25)
        #self.dateBox1.resize(80, 22)
        self.dateBox1.valueChanged.connect(self.dateBoxChanged)
        self.dateBox2 = QSpinBox(self)
        self.dateBox2.setMaximum(20201201)
        # self.dateBox2.move(80, 50)
        #self.dateBox2.resize(80, 22)
        self.dateBox2.valueChanged.connect(self.dateBoxChanged)

        # 크롤링 버튼
        btn1 = QPushButton("Crawling", self)
        # 다이얼로그 버튼
        btn2 = QPushButton('Bootstrapping', self)
        # btn1.move(10, 100)
        btn1.clicked.connect(self.kofiabondCrawling)
        btn2.clicked.connect(self.BootstrappingPage)

        self.tableWidget0 = QTableWidget(self)
        self.tableWidget0.setEditTriggers(QAbstractItemView.NoEditTriggers)
        self.tableWidget0.setHorizontalHeaderLabels(['Maturity', 'YTMs'])

        # 레이아웃 세팅
        leftInnerLayOut1 = QVBoxLayout()
        leftInnerLayOut1.addWidget(label1)
        leftInnerLayOut1.addWidget(label2)
        leftInnerLayOut2 = QVBoxLayout()
        leftInnerLayOut2.addWidget(self.dateBox1)
        leftInnerLayOut2.addWidget(self.dateBox2)
        leftInnerLayOut3 = QHBoxLayout()
        leftInnerLayOut3.addLayout(leftInnerLayOut1)
        leftInnerLayOut3.addLayout(leftInnerLayOut2)
        
        leftInnerLayOut = QVBoxLayout()
        leftInnerLayOut.addLayout(leftInnerLayOut3)
        leftInnerLayOut.addWidget(btn1)
        leftInnerLayOut.addWidget(btn2)
        
        # 그룹박스안에 레이아웃 세팅
        groupBox.setLayout(leftInnerLayOut)
        #오른쪽 레이아웃
        #rightLayout = QVBoxLayout()
        #rightLayout.addWidget(self.tableWidget)

        # 양쪽 레이아웃 평행세팅
        layout = QHBoxLayout()
        layout.addWidget(groupBox)
        layout.addWidget(self.tableWidget0)

        # 전체세팅
        self.setLayout(layout)

    # 버튼슬롯 -> <과제1> crawling method 구현해야함.
    def kofiabondCrawling(fr_date,to_date):

        #
        #   크롤링
        #

        col_dt = {
            'standardDt': 'date',
            'val2': 'avg',
            'val3': 'nic',
            'val4': 'kor',
            'val5': 'kis',
            'val6': 'fnp',
        }

        col_order = [
            'date', 'bnd_cd', 'mat_cd', 'avg', 'nic', 'kor', 'kis', 'fnp'
        ]
        
        df_all=pd.DataFrame()

        for bnd_type in list(bnd_df['bond_cd'])[:2]:  # 첫번째 채권(국채/국고채)에 대해서만
            for mat_type in list(mat_df['mat_cd']):  # 모든 만기에 대해(0.5~50년)

                # 조회하기
                form_data_src = '''
                                    <message>
                                      <proframeHeader>
                                        <pfmAppName>BIS-KOFIABOND</pfmAppName>
                                        <pfmSvcName>BISBndSrtPrcSrchSO</pfmSvcName>
                                        <pfmFnName>selectTrm</pfmFnName>
                                      </proframeHeader>
                                      <systemHeader></systemHeader>
                                        <BISBndSrtPrcTrmDTO>
                                        <val31>bnd_cd_var</val31>
                                        <val32>mat_cd_var</val32>
                                        <creditEstOrgCd>comp_cd_var</creditEstOrgCd>
                                        <standardDt1>fr_date_var</standardDt1>
                                        <standardDt2>to_date_var</standardDt2>
                                    </BISBndSrtPrcTrmDTO>
                                    </message>
                                    '''

                form_data_src = form_data_src.replace('bnd_cd_var', bnd_type) \
                    .replace('mat_cd_var', mat_type) \
                    .replace('comp_cd_var', '0') \
                    .replace('fr_date_var', str(fr_date)) \
                    .replace('to_date_var', str(to_date))

                r = requests.post(gen_req_url, form_data_src)
                soup = BeautifulSoup(r.content, 'lxml-xml')
                res_li = soup.find_all('BISBndSrtPrcTrmDTO')

                df = pd.DataFrame()
                for r in res_li:
                    dt = dict()
                    for i in list(col_dt.keys()):
                        dt[i] = [r.find(i).text]
                    df_sub = pd.DataFrame.from_dict(dt)
                    df = df.append(df_sub, sort=True)

                df['bnd_cd'] = bnd_type
                df['mat_cd'] = mat_type
                df_all.append(df, sort=True)
                print(bnd_type, mat_type, fr_date, to_date)

        df_all.reset_index(drop=True, inplace=True)
        df_all = df_all.rename(columns=col_dt)
        # 문자형 숫자 -> 숫자형 숫자로
        col_numeric = ['date', 'avg', 'nic', 'kor', 'kis', 'fnp']
        for i in col_numeric:
            df_all[i] = pd.to_numeric(df_all[i], errors='coerce')
        df_all = df_all.reindex(columns=col_order)
        
        #데이타 프레임의 날짜 리스트 뽑아오기
        date_list = list(df_all['date'])
        #date 컬럼 삭제
        del df_all['date']
        #데이타 프레임의 인덱스를 date_list로 설정
        df_all.index = date_list
        df_god = pd.DataFrame()
        df_god=df_all[['mat_cd','avg']]
        df_god['mat_cd']=pd.to_numeric(df_god['mat_cd'])

        #반복문으로 해결하기
        mat = '3 6 9 100 106 200 206 300 400 500 700 1000 2000 3000 5000'
        mat_list = mat.split(' ')
        
        total_df =pd.DataFrame()
        for i,mat in enumerate(mat_list):
            temp_df=df_god[df_god['mat_cd']==int(mat)]
            del temp_df['mat_cd']
            temp_df.columns = [str(mat)]
            if i != 0:
                    total_df = pd.merge(total_df, temp_df, how='outer', right_index=True, left_index=True)
            else:
                    total_df = temp_df

        total_df.sort_index(ascending = False)
        #컬럼명 변경해주기
        new_mat = '0.25 0.5 0.75 1 1.5 2 2.5 3 4 5 7 10 20 30 50'
        new_mat_list = new_mat.split(' ')
        total_df.columns = new_mat_list
        final_df = total_df.sort_index(ascending = False)

        YTMs = final_df.iloc[0].values[:12]
        Bond_data = {'Maturity': ["%s" % Maturity[0], "%s" % Maturity[1], "%s" % Maturity[2], "%s" % Maturity[3],
                                  "%s" % Maturity[4], "%s" % Maturity[5], "%s" % Maturity[6], "%s" % Maturity[7],
                                  "%s" % Maturity[8], "%s" % Maturity[9], "%s" % Maturity[10], "%s" % Maturity[11]],
                     'YTMs': ["%s" % YTMs[0], "%s" % YTMs[1], "%s" % YTMs[2], "%s" % YTMs[3], "%s" % YTMs[4],
                              "%s" % YTMs[5], "%s" % YTMs[6], "%s" % YTMs[7], "%s" % YTMs[8], "%s" % YTMs[9],
                              "%s" % YTMs[10], "%s" % YTMs[11]]
                     }
        column_idx_lookup = {'Maturity': 0, 'YTMs': 1}

        for k, v in Bond_data.items():
            col = column_idx_lookup[k]
            for row, val in enumerate(v):
                item = QTableWidgetItem(val)

                self.tableWidget0.setItem(row, col, item)

        self.tableWidget0.resizeColumnsToContents()
        self.tableWidget0.resizeRowsToContents()

    def BootstrappingPage(self):
        column_idx_lookup_2 = {'Zerorates': 2}
        ###########  Bootstrap 코드 ###########
        number = 1
        # Maturity = array(data.loc[0,'3월':])

        # a = pd.DataFrame()
        # a['mat'] = Maturity

        B = []
        for i in range(1, 41):
            B.append(0.25 * i)
        b = pd.DataFrame(B)
        C = []
        for i in range(1, 121):
            C.append(i)
        c = pd.DataFrame(C)

        for r in range(1, number + 1):
            # YTMs = data.loc[r,'3월':]/100
            zeros = np.zeros(40)
            tzeros = np.zeros(40)
            zerorates = np.zeros(12)
            for i in range(len(zeros)):
                zeros[i] = -1
                tzeros[i] = zeros[i]
            zerorates[0] = np.log(1 + YTMs[0] * Maturity[0]) / Maturity[0]
            zeros[0] = zerorates[0]
            tzeros[0] = zeros[0]

            def Par_value(seed, N):
                tmpPrice = 0
                numbuck = int(Maturity[N] / 0.25)
                tzeros[numbuck - 1] = seed
                for i in range(0, numbuck - 1):
                    if zeros[i] < 0:
                        tzeros[i] = (0.25 * tzeros[numbuck - 1] + (Maturity[N] - 0.25 * (i + 1)) * tzeros[i - 1]) / (
                                    Maturity[N] - 0.25 * i)
                        tmpPrice = tmpPrice + (YTMs[N] * 0.25) * exp(-tzeros[i] * 0.25 * (i + 1))
                    else:
                        tmpPrice = tmpPrice + (YTMs[N] * 0.25) * exp(-zeros[i] * 0.25 * (i + 1))
                tmpPrice2 = tmpPrice + (1 + YTMs[N] * 0.25) * exp(-seed * Maturity[N])
                return tmpPrice2 - 1

            for i in range(1, 12):
                optimize.newton(Par_value, 0, args=(i,))
                zerorates[i] = optimize.newton(Par_value, 0, args=(i,))
                for j in range(0, 40):
                    zeros[j] = tzeros[j]
            b[r] = zeros
            # for i in range(0,12):
            #     a.loc[i,r] = zerorates[i]
            # a = a.rename(columns={(r):data.loc[r,'일자']})
        a = zerorates

        # x = [b1,b2,b3,b4,a1,a2]
        f = lambda k, M: (k[0]) + (k[1] * ((1 - exp(- M / k[4])) / (M / k[4]))) + (
                    k[2] * (((1 - exp(-M / k[4])) / (M / k[4])) - (exp(-M / k[4])))) + (
                                     k[3] * (((1 - exp(-M / k[5])) / (M / k[5])) - (exp(-M / k[5]))))

        def error(x, M, y_data):
            if x[0] + x[1] > 0 and x[0] > 0:
                return ((f(x, M) - y_data) ** 2).sum()
            else:
                return 1000  # NSS Model과 error func 함수 설정

        M = array(Maturity)
        y_data = array(zerorates)
        x0 = array([0.01, 0.01, 0.01, 0.01, 1, 1])  # 주어진 데이터 입력
        x = optimize.fmin(error, x0, args=(M, y_data,))  # error func의 최솟값
        m = x

        dlg = BootstrappingDialog()
        dlg.exec_()

        # 조회일설정 슬롯

    def dateBoxChanged(self):
        fr_date = str(self.dateBox1.value())
        to_date = str(self.dateBox2.value())


# 루프실행
if __name__ == "__main__":
    app = QApplication(sys.argv)
    mywindow = MyWindow()
    mywindow.show()
    app.exec_()

1010000 0003 <__main__.MyWindow object at 0x000000010B0DBB88> False
1010000 0006 <__main__.MyWindow object at 0x000000010B0DBB88> False
1010000 0009 <__main__.MyWindow object at 0x000000010B0DBB88> False
1010000 0100 <__main__.MyWindow object at 0x000000010B0DBB88> False
1010000 0106 <__main__.MyWindow object at 0x000000010B0DBB88> False
1010000 0200 <__main__.MyWindow object at 0x000000010B0DBB88> False
1010000 0206 <__main__.MyWindow object at 0x000000010B0DBB88> False
1010000 0300 <__main__.MyWindow object at 0x000000010B0DBB88> False
1010000 0400 <__main__.MyWindow object at 0x000000010B0DBB88> False
1010000 0500 <__main__.MyWindow object at 0x000000010B0DBB88> False
1010000 0700 <__main__.MyWindow object at 0x000000010B0DBB88> False
1010000 1000 <__main__.MyWindow object at 0x000000010B0DBB88> False
1010000 2000 <__main__.MyWindow object at 0x000000010B0DBB88> False
1010000 3000 <__main__.MyWindow object at 0x000000010B0DBB88> False
1010000 5000 <__main__.MyWindow object at 0x0000

KeyError: 'date'